In [2]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
from keras import optimizers
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.models import load_model

from copy import copy
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [3]:
import sys
#sys.path.insert(0, '../../')
sys.path.insert(0, '/home/luka/Developement/accetuation/')
from prepare_data import *

%run ../../prepare_data.py

In [27]:
data = Data('sl', accent_classification=True, reverse_inputs=False)
data.generate_data('syllables_accent_classification_correct_input_order_train',
                   'syllables_accent_classification_correct_input_order_test',
                   'syllables_accent_classification_correct_input_order_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

LOADING DATA...
LOAD SUCCESSFUL!


In [36]:
data = Data('sl', accent_classification=True)
data.generate_data('syllables_accent_classification_train',
                   'syllables_accent_classification_test',
                   'syllables_accent_classification_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

LOADING DATA...
LOAD SUCCESSFUL!


In [12]:
data = Data('s', accent_classification=True, reverse_inputs=False)
data.generate_data('syllables_accent_classification_correct_input_order_train',
                   'syllables_accent_classification_correct_input_order_test',
                   'syllables_accent_classification_correct_input_order_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [4]:
data = Data('s', accent_classification=True)
data.generate_data('syllables_accent_classification_train',
                   'syllables_accent_classification_test',
                   'syllables_accent_classification_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/', test_set=True)

LOADING DATA...
LOAD SUCCESSFUL!


In [74]:
data = Data('l', accent_classification=True, reverse_inputs=False)
data.generate_data('letters_accent_classification_correct_input_order_train',
                   'letters_accent_classification_correct_input_order_test',
                   'letters_accent_classification_correct_input_order_validate',
                   inputs_location='../internal_representations/inputs/', content_location='../../data/')

LOADING DATA...
LOAD SUCCESSFUL!


In [3]:
data = Data('l', accent_classification=True)
data.generate_data('letters_accent_classification_train',
                   'letters_accent_classification_test',
                   'letters_accent_classification_validate', content_name='SlovarIJS_BESEDE_utf8.lex',
                      content_shuffle_vector='content_shuffle_vector', shuffle_vector='shuffle_vector',
                      inputs_location='../internal_representations/inputs/', content_location='../../data/')

LOADING DATA...
LOAD SUCCESSFUL!


In [12]:
#data.x_train = None
content = data._read_content('../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [6]:
len(syllable_dictionary)

5168

In [5]:
data.x_train = np.concatenate((data.x_train, data.x_test), axis=0)
data.x_other_features_train = np.concatenate((data.x_other_features_train, data.x_other_features_test), axis=0)
data.y_train = np.concatenate((data.y_train, data.y_test), axis=0)

In [75]:
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
# actual_epoch = 1
actual_epoch = 20
# num_fake_epoch = 2
num_fake_epoch = 20



# letters
conv_input_shape=(23, 36)

# syllabled letters
# conv_input_shape=(10, 5168)
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)


othr_input = Input(shape=othr_input, name='othr_input')

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [76]:
model.load_weights('letters/v3_0/20_test_epoch.h5')
#model = load_model('syllabled_letters/v2_5_3/40_epoch.h5', custom_objects={'actual_accuracy': actual_accuracy})

In [17]:
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
# actual_epoch = 1
actual_epoch = 20
# num_fake_epoch = 2
num_fake_epoch = 20



# letters
# conv_input_shape=(23, 36)

# syllabled letters
conv_input_shape=(10, 5168)


othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# letters
# x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
# x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)

# syllabled letters
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [18]:
#model.load_weights('syllables/v2_0/20_test_epoch.h5')
model.load_weights('syllables/v2_2/20_test_epoch.h5')

In [28]:
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
# actual_epoch = 1
actual_epoch = 20
# num_fake_epoch = 2
num_fake_epoch = 20



# letters
# conv_input_shape=(23, 36)

# syllabled letters
conv_input_shape=(10, 252)


othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
# letters
# x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
# x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)

# syllabled letters
x_conv = Conv1D(200, (2), padding='same', activation='relu')(conv_input)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])

In [30]:
model.load_weights('syllabled_letters/v2_2/20_test_epoch.h5')

In [55]:
#95.54408764380132
#95.31371030209141
data._reverse_inputs = True

In [31]:
y_array = np.asarray(data.y_validate)
accentuation_length = (y_array > 0).sum()
predictions = model.predict_generator(data.generator('validate', batch_size, content_name='SlovarIJS_BESEDE_utf8.lex', content_location='../../data/'), 
                                      accentuation_length/(batch_size))

In [37]:
accuracy, real_accuracy, errors = data.test_type_accuracy(predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print(accuracy)
print(real_accuracy)

58327
54222
51745
2477
54222
95.6589572582166
95.43174357271957


In [64]:
predictions.shape

(58327, 13)

In [33]:
def reorder_correct_direction_inputs(predictions, y):
    pred_i = 0
    for i in range(len(y)):
        num_accented_syllables = 0
        for el in y[i]:
            if el > 0:
                num_accented_syllables += 1
        if num_accented_syllables > 1:
            min_i = pred_i
            max_i = pred_i + num_accented_syllables - 1
            while(max_i > min_i):
                min_pred = copy(predictions[min_i])
                max_pred = copy(predictions[max_i])
                predictions[min_i] = max_pred
                predictions[max_i] = min_pred
                min_i += 1
                max_i -= 1
        pred_i += num_accented_syllables
        
reorder_correct_direction_inputs(predictions, data.y_validate)

In [35]:
output = open('error_analysis_results/onedirectional_input/syllabled_letters_accent_classification_co_oversampling_error.pkl', 'wb')
pickle.dump(errors, output)
output.close()
output = open('error_analysis_results/onedirectional_input/syllabled_letters_accent_classification_co_oversampling_predictions.pkl', 'wb')
pickle.dump(predictions, output)
output.close()

In [53]:
pickle_input = open('error_analysis_results/onedirectional_input/letters_accent_classification_correct_order_reversed_predictions.pkl', 'rb')
letters_accent_classification_correct_order_reversed_predictions = pickle.load(pickle_input)
pickle_input = open('error_analysis_results/onedirectional_input/syllables_accent_classification_correct_order_reversed_predictions.pkl', 'rb')
syllables_accent_classification_correct_order_reversed_predictions = pickle.load(pickle_input)
pickle_input = open('error_analysis_results/onedirectional_input/syllabled_letters_accent_classification_correct_order_reversed_predictions.pkl', 'rb')
syllabled_letters_accent_classification_correct_order_reversed_predictions = pickle.load(pickle_input)

pickle_input = open('letters_accent_classification_test_predictions.pkl', 'rb')
letters_accent_classification_predictions = pickle.load(pickle_input)
pickle_input = open('syllables_accent_classification_test_predictions.pkl', 'rb')
syllables_accent_classification_predictions = pickle.load(pickle_input)
pickle_input = open('syllabled_letters_accent_classification_test_predictions.pkl', 'rb')
syllabled_letters_accent_classification_predictions = pickle.load(pickle_input)

In [55]:
ensemble_predictions = np.mean( np.array([ letters_accent_classification_predictions, syllables_accent_classification_predictions, syllabled_letters_accent_classification_predictions,
                                         letters_accent_classification_correct_order_reversed_predictions, syllables_accent_classification_correct_order_reversed_predictions, syllabled_letters_accent_classification_correct_order_reversed_predictions]), axis=0 )

#ensemble_predictions = np.mean( np.array([ letters_accent_classification_predictions, syllabled_letters_accent_classification_predictions,
#                                         letters_accent_classification_correct_order_reversed_predictions, syllabled_letters_accent_classification_correct_order_reversed_predictions]), axis=0 )

#ensemble_predictions = np.mean( np.array([ letters_accent_classification_predictions, syllables_accent_classification_predictions, syllabled_letters_accent_classification_predictions ]), axis=0 )
accuracy, real_accuracy, errors = data.test_type_accuracy(ensemble_accent_classification_predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
# accuracy, real_accuracy, errors = data.test_type_accuracy(ensemble_predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)
print(accuracy)
print(real_accuracy)

58327
54222
2837
51385
54222
3.228350506626434
5.23219357456383


In [83]:
print(data.y_validate[25])
print(predictions[25])
print(predictions[26])
print(letters_accent_classification_predictions[25])
print(letters_accent_classification_predictions[26])
print(real_accuracy)

[ 4.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
[  0.00000000e+00   1.00000000e+00   4.25894693e-12   1.31152606e-21
   7.07428871e-37   6.76180402e-29   6.22833575e-15   0.00000000e+00
   5.17264047e-22   0.00000000e+00   0.00000000e+00   2.81393113e-18
   0.00000000e+00]
[  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.51565325e-25
   1.00000000e+00   1.61675484e-34   6.17491277e-39   0.00000000e+00
   2.99232293e-38   0.00000000e+00   4.22588520e-34   0.00000000e+00
   0.00000000e+00]
[  0.00000000e+00   9.99982595e-01   9.06433911e-07   3.11275741e-16
   3.69056970e-18   1.20418770e-17   1.11667652e-12   2.57631544e-25
   1.55146012e-17   0.00000000e+00   2.08245277e-24   0.00000000e+00
   0.00000000e+00]
[  0.00000000e+00   2.67856488e-26   4.35386780e-13   1.66966936e-17
   1.00000000e+00   1.27324307e-27   6.43282027e-28   5.31304462e-32
   1.08525564e-34   0.00000000e+00   7.42423605e-29   0.00000000e+00
   0.00000000e+00]
95.5036700970086


In [80]:
print(data.y_validate[25])
print(predictions[25])
print(predictions[26])
print(letters_accent_classification_predictions[25])
print(letters_accent_classification_predictions[26])
print(real_accuracy)

[ 4.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
[  0.00000000e+00   0.00000000e+00   0.00000000e+00   1.51565325e-25
   1.00000000e+00   1.61675484e-34   6.17491277e-39   0.00000000e+00
   2.99232293e-38   0.00000000e+00   4.22588520e-34   0.00000000e+00
   0.00000000e+00]
[  0.00000000e+00   1.00000000e+00   4.25894693e-12   1.31152606e-21
   7.07428871e-37   6.76180402e-29   6.22833575e-15   0.00000000e+00
   5.17264047e-22   0.00000000e+00   0.00000000e+00   2.81393113e-18
   0.00000000e+00]
[  0.00000000e+00   9.99982595e-01   9.06433911e-07   3.11275741e-16
   3.69056970e-18   1.20418770e-17   1.11667652e-12   2.57631544e-25
   1.55146012e-17   0.00000000e+00   2.08245277e-24   0.00000000e+00
   0.00000000e+00]
[  0.00000000e+00   2.67856488e-26   4.35386780e-13   1.66966936e-17
   1.00000000e+00   1.27324307e-27   6.43282027e-28   5.31304462e-32
   1.08525564e-34   0.00000000e+00   7.42423605e-29   0.00000000e+00
   0.00000000e+00]
95.5036700970086


In [76]:
accuracy, errors = data.test_accuracy(new_predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, syllable_dictionary=syllable_dictionary)
print(accuracy)

86.9573235955885


In [25]:
predictions[1]

array([  1.30016403e-07,   9.99997973e-01,   2.21242431e-14,
         4.04031276e-34,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00], dtype=float32)

In [47]:
predictions[1]

array([  1.30016403e-07,   9.99997973e-01,   2.21242431e-14,
         4.04031276e-34,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00], dtype=float32)

In [26]:
new_predictions[1]

array([  2.21242431e-14,   9.99997973e-01,   1.30016403e-07,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
         0.00000000e+00], dtype=float32)

In [60]:
def get_word_length(x_el):
    i = 0
    for el in x_el:
        if el == 0:
            return i
        i += 1

new_predictions = np.zeros(predictions.shape, dtype='float32')
for i in range(len(predictions)):
    word_len = get_word_length(data.x_test[i])
    #print(word_len)
    #if word_len > 6:
    #    print(word_len)
    #    print(data.y_test[i])
    
    #
    for k in range(word_len):
        new_predictions[i][k] += predictions[i][word_len - 1 - k]

        
accuracy, errors = data.test_accuracy(new_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, syllable_dictionary=syllable_dictionary)
print(accuracy)
        
errors = [[el[0], el[1][::-1], el[2], el[3][::-1], el[4][::-1]] for el in errors]


errors.sort(key=lambda x: x[1])
# name = 'error_analysis_results/bidirectional_input/letters_word_accetuation'
# name = 'error_analysis_results/bidirectional_input/letters_word_accetuation_test'
# name = 'error_analysis_results/bidirectional_input/syllables_word_accetuation'
# name = 'error_analysis_results/bidirectional_input/syllables_word_accetuation_test'
# name = 'error_analysis_results/bidirectional_input/syllabled_letters_word_accetuation'
#name = 'error_analysis_results/bidirectional_input/syllabled_letters_word_accetuation_test'
# name = 'error_analysis_results/onedirectional_input/letters_word_accetuation_correct_order_reversed_test'
# name = 'error_analysis_results/onedirectional_input/syllabled_letters_word_accetuation_correct_order_reversed_test'
# name = 'error_analysis_results/onedirectional_input/syllables_word_accetuation_correct_order_reversed_test'
# name = 'error_analysis_results/onedirectional_input/letters_word_accetuation_correct_order_reversed'
# name = 'error_analysis_results/onedirectional_input/syllabled_letters_word_accetuation_correct_order_reversed'
# name = 'error_analysis_results/onedirectional_input/syllables_word_accetuation_correct_order_reversed'
output = open(name + '_error.pkl', 'wb')
pickle.dump(errors, output)
output.close()
output = open(name + '_predictions.pkl', 'wb')
pickle.dump(new_predictions, output)
output.close()

88.73564101579008


In [10]:
# TEST DATA
#pickle_input = open('letters_word_accetuation_test_error.pkl', 'rb')
#letters_word_accetuation_test = pickle.load(pickle_input)
#pickle_input = open('syllables_word_accetuation_test_error.pkl', 'rb')
#syllables_word_accetuation_test = pickle.load(pickle_input)
#pickle_input = open('syllabled_letters_word_accetuation_test_error.pkl', 'rb')
#syllabled_letters_word_accetuation_test = pickle.load(pickle_input)
#pickle_input = open('ensemble_test_errors.pkl', 'rb')
#ensemble_word_accetuation_test = pickle.load(pickle_input)

pickle_input = open('letters_word_accetuation_predictions.pkl', 'rb')
letters_word_accetuation_predictions = pickle.load(pickle_input)
pickle_input = open('syllables_word_accetuation_predictions.pkl', 'rb')
syllables_word_accetuation_predictions = pickle.load(pickle_input)
pickle_input = open('syllabled_letters_word_accetuation_predictions.pkl', 'rb')
syllabled_letters_word_accetuation_predictions = pickle.load(pickle_input)


pickle_input = open('error_analysis_results/onedirectional_input/letters_word_accetuation_correct_order_reversed_predictions.pkl', 'rb')
letters_word_accetuation_correct_order_predictions = pickle.load(pickle_input)
pickle_input = open('error_analysis_results/onedirectional_input/syllabled_letters_word_accetuation_correct_order_reversed_predictions.pkl', 'rb')
syllables_word_accetuation_correct_order_predictions = pickle.load(pickle_input)
pickle_input = open('error_analysis_results/onedirectional_input/syllables_word_accetuation_correct_order_reversed_predictions.pkl', 'rb')
syllabled_letters_word_accetuation_correct_order_predictions = pickle.load(pickle_input)

#pickle_input = open('svm/rf_predictions.pkl', 'rb')
#rf_predictions = pickle.load(pickle_input)


# !!!!!!!!! TEST
#pickle_input = open('letters_word_accetuation_test_predictions.pkl', 'rb')
#letters_word_accetuation_predictions = pickle.load(pickle_input)
#pickle_input = open('syllables_word_accetuation_test_predictions.pkl', 'rb')
#syllables_word_accetuation_predictions = pickle.load(pickle_input)
#pickle_input = open('syllabled_letters_word_accetuation_test_predictions.pkl', 'rb')
#syllabled_letters_word_accetuation_predictions = pickle.load(pickle_input)


#pickle_input = open('error_analysis_results/onedirectional_input/letters_word_accetuation_correct_order_reversed_test_predictions.pkl', 'rb')
#letters_word_accetuation_correct_order_predictions = pickle.load(pickle_input)
#pickle_input = open('error_analysis_results/onedirectional_input/syllabled_letters_word_accetuation_correct_order_reversed_test_predictions.pkl', 'rb')
#syllables_word_accetuation_correct_order_predictions = pickle.load(pickle_input)
#pickle_input = open('error_analysis_results/onedirectional_input/syllables_word_accetuation_correct_order_reversed_test_predictions.pkl', 'rb')
#syllabled_letters_word_accetuation_correct_order_predictions = pickle.load(pickle_input)

#pickle_input = open('svm/rf_predictions.pkl', 'rb')
#rf_predictions = pickle.load(pickle_input)

In [70]:
ensemble_predictions = np.mean( np.array([ letters_word_accetuation_predictions, syllables_word_accetuation_predictions, syllabled_letters_word_accetuation_predictions, 
                                         letters_word_accetuation_correct_order_predictions, syllables_word_accetuation_correct_order_predictions, syllabled_letters_word_accetuation_correct_order_predictions]), axis=0 )
#ensemble_predictions = np.mean( np.array([ letters_word_accetuation_predictions, syllables_word_accetuation_predictions, syllabled_letters_word_accetuation_predictions]), axis=0 )
#accuracy, errors = data.test_accuracy(ensemble_predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, syllable_dictionary=syllable_dictionary)
ensemble_predictions=syllabled_letters_word_accetuation_correct_order_predictions
accuracy, errors = data.test_accuracy(ensemble_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, syllable_dictionary=syllable_dictionary, threshold=0.5)
print(accuracy)

87.21041914787352


In [23]:
ensemble_predictions = np.mean( np.array([ letters_word_accetuation_predictions, syllables_word_accetuation_predictions, syllabled_letters_word_accetuation_predictions, 
                                         letters_word_accetuation_correct_order_predictions, syllables_word_accetuation_correct_order_predictions, syllabled_letters_word_accetuation_correct_order_predictions]), axis=0 )
#ensemble_predictions = np.mean( np.array([ letters_word_accetuation_predictions, syllables_word_accetuation_predictions, syllabled_letters_word_accetuation_predictions]), axis=0 )
#accuracy, errors = data.test_accuracy(ensemble_predictions, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, syllable_dictionary=syllable_dictionary)
accuracy, errors = data.test_accuracy(ensemble_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, syllable_dictionary=syllable_dictionary, threshold=0.1)
print(accuracy)

71.62779976180414


In [ ]:
accuracy, errors = data.test_accuracy(ensemble_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, syllable_dictionary=syllable_dictionary, threshold=0.1)
print(accuracy)

In [19]:
ensemble_predictions[0][0]

1.6609331e-10

In [56]:
errors = [[el[0], el[1][::-1], el[2], el[3][::-1], el[4][::-1]] for el in errors]
errors.sort(key=lambda x: x[1])

output = open('error_analysis_results/onedirectional_input/ensemble_errors.pkl', 'wb')
pickle.dump(errors, output)
output.close()
output = open('error_analysis_results/onedirectional_input/ensemble_predictions.pkl', 'wb')
pickle.dump(ensemble_predictions, output)
output.close()

In [84]:
len(predictions)

52058

In [19]:
def compare_outputs(outputs1, outputs2):
    outputs2_indices = [el[0] for el in outputs2]
    return [el for el in outputs1 if el[0] in outputs2_indices]

def get_unique_errors(outputs1, outputs2):
    outputs2_indices = [el[0] for el in outputs2]
    return [el for el in outputs1 if el[0] not in outputs2_indices]

print(len(letters_word_accetuation))
print(len(syllables_word_accetuation))
print(len(syllabled_letters_word_accetuation))
print(len(compare_outputs(letters_word_accetuation, syllables_word_accetuation)))
print(len(compare_outputs(letters_word_accetuation, syllabled_letters_word_accetuation)))
print(len(compare_outputs(syllables_word_accetuation, syllabled_letters_word_accetuation)))


print(1  - 7073.0/52058)
collective_errors = compare_outputs(letters_word_accetuation, compare_outputs(syllabled_letters_word_accetuation, syllables_word_accetuation))
print(len(collective_errors))

# print(letters_word_accetuation[:20])
# print(syllables_word_accetuation[:20])
# print(syllabled_letters_word_accetuation[:20])
# print(collective_errors)

# unique_err = get_unique_errors(get_unique_errors(letters_word_accetuation, syllables_word_accetuation), syllabled_letters_word_accetuation)
# unique_err = get_unique_errors(get_unique_errors(syllables_word_accetuation, letters_word_accetuation), syllabled_letters_word_accetuation)
unique_err = get_unique_errors(get_unique_errors(syllabled_letters_word_accetuation, letters_word_accetuation), syllables_word_accetuation)
print("UNIQUE ERRORS")
print(len(unique_err))

6214
7770
6515
3232
3662
3868
0.8641323139575089
2609
UNIQUE ERRORS
1594


In [100]:
print(letters_word_accetuation_predictions[0])
print(syllables_word_accetuation_predictions[0])
print(syllabled_letters_word_accetuation_predictions[0])

test = np.mean( np.array([ letters_word_accetuation_predictions, syllables_word_accetuation_predictions, syllabled_letters_word_accetuation_predictions ]), axis=0 )
print(test[0])
print((9.99999106e-01 + 9.99989212e-01 + 1.00000000e+00)/3)

[  1.84468085e-08   9.99999106e-01   2.88171825e-17   7.48493946e-24
   4.87220418e-36   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00]
[  3.23229117e-11   9.99989212e-01   2.31212027e-08   2.49620165e-26
   3.08989440e-20   4.69045294e-21   4.73113282e-38   1.19897549e-33
   8.32371040e-34   0.00000000e+00]
[  4.11718755e-27   1.00000000e+00   2.89068776e-27   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00]
[  6.15971052e-09   9.99996126e-01   7.70706787e-09   2.50330040e-24
   1.02996483e-20   1.56348435e-21   1.57704427e-38   3.99658482e-34
   2.77457006e-34   0.00000000e+00]
0.9999961059999999


In [12]:
ensemble_predictions = np.mean( np.array([ letters_word_accetuation_predictions, syllables_word_accetuation_predictions, syllabled_letters_word_accetuation_predictions ]), axis=0 )
accuracy, errors = data.test_accuracy(ensemble_predictions, data.x_test, data.x_other_features_test, data.y_test, dictionary, feature_dictionary, vowels, syllable_dictionary=syllable_dictionary)
print(accuracy)

91.7054055092397


In [54]:
accuracy

91.7054055092397

In [79]:
pickle_input = open('error_analysis_results/onedirectional_input/ensemble_predictions.pkl', 'rb')
ensemble_accent_classification_predictions = pickle.load(pickle_input)

In [70]:
pickle_input = open('../accent_classification/syllabled_letters_accent_classification_test_predictions.pkl', 'rb')
syllables_word_accetuation_predictions = pickle.load(pickle_input)
ensemble = syllables_word_accetuation_predictions

In [73]:
pickle_input = open('error_analysis_results/onedirectional_input/syllabled_letters_accent_classification_co_oversampling_predictions.pkl', 'rb')
syllables_word_accetuation_predictions = pickle.load(pickle_input)
ensemble = syllables_word_accetuation_predictions

In [77]:
accuracy, real_accuracy, errors = data.test_type_accuracy(ensemble, data.x_validate, data.x_other_features_validate, data.y_validate, dictionary, feature_dictionary, vowels, accented_vowels, syllable_dictionary=syllable_dictionary)

58327
54222
51745
2477
54222


In [66]:
real_accuracy

95.98318025893549

In [78]:
real_accuracy

95.43174357271957

In [80]:


probabilities = np.zeros((13, 13))
count_results = np.zeros(13)
l = 0
max_indices = []
for i in range(len(data.y_validate)):
    decoded_x = data.decode_syllable_x(data.x_validate[i], syllable_dictionary)
    for j in range(10):
        if data.y_validate[i][j] > 0.0:
            
            
            stressed_letter = data.get_accentuated_letter(decoded_x, j, vowels, syllables=data._input_type != 'l')
            possible_places = np.zeros(len(ensemble[l]))
            if stressed_letter == 'r':
                possible_places[0] = 1
            elif stressed_letter == 'a':
                possible_places[1] = 1
                possible_places[2] = 1
            elif stressed_letter == 'e':
                possible_places[3] = 1
                possible_places[4] = 1
                possible_places[5] = 1
            elif stressed_letter == 'i':
                possible_places[6] = 1
                possible_places[7] = 1
            elif stressed_letter == 'o':
                possible_places[8] = 1
                possible_places[9] = 1
                possible_places[10] = 1
            elif stressed_letter == 'u':
                possible_places[11] = 1
                possible_places[12] = 1
            possible_predictions = ensemble[l] * possible_places
            
            
            max_index = np.argmax(possible_predictions)
            possible_predictions[max_index] = 0.0
            probabilities[int(data.y_validate[i][j])] += possible_predictions
            count_results[int(data.y_validate[i][j])] += 1
            l += 1
            
for i in range(len(probabilities)):
    probabilities[i] = probabilities[i] / count_results[i]

In [66]:
type(ensemble_accent_classification_predictions)

numpy.ndarray

In [67]:
type(possible_predictions)

numpy.ndarray

In [81]:
print(np.around(probabilities, decimals=2))
print(count_results.astype(int))

[[  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
    nan]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.01  0.    0.01  0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.01  0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.01  0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.02  0.01  0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.06  0.03  0.11  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   

In [72]:
print(np.around(probabilities, decimals=2))
print(count_results.astype(int))

[[  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
    nan]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.01  0.    0.01  0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.01  0.01  0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.02  0.02  0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.01  0.01  0.03  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   

In [76]:
print(np.around(probabilities, decimals=2))
print(count_results.astype(int))

[[  nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan   nan
    nan]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.01  0.    0.01  0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.01  0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.01  0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.02  0.01  0.    0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.06  0.03  0.11  0.    0.  ]
 [ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.   

In [48]:
data._get_accented_vowels()

['à', 'á', 'ä', 'é', 'ë', 'ì', 'í', 'î', 'ó', 'ô', 'ö', 'ú', 'ü']

In [46]:
print(data.x_test[0])
print(get_word_length(data.x_test[0]))

[1359  834    0    0    0    0    0    0    0    0]
2


In [ ]:
#######################################################################################
# COLLECTIVE RESULTS
#######################################################################################

In [16]:
# TEST DATA!!!
pickle_input = open('../accent_classification/letters_accent_classification_test_predictions.pkl', 'rb')
letters_accent_classification_predictions = pickle.load(pickle_input)
pickle_input = open('../accent_classification/syllables_accent_classification_test_predictions.pkl', 'rb')
syllables_accent_classification_predictions = pickle.load(pickle_input)
pickle_input = open('../accent_classification/syllabled_letters_accent_classification_test_predictions.pkl', 'rb')
syllabled_letters_accent_classification_predictions = pickle.load(pickle_input)
pickle_input = open('../accent_classification/ensemble_test_predictions.pkl', 'rb')
ensemble_accent_classification_predictions = pickle.load(pickle_input)

pickle_input = open('../accent_classification/letters_accent_classification_test_errors.pkl', 'rb')
letters_accent_classification_errors = pickle.load(pickle_input)
pickle_input = open('../accent_classification/syllables_accent_classification_test_errors.pkl', 'rb')
syllables_accent_classification_errors = pickle.load(pickle_input)
pickle_input = open('../accent_classification/syllabled_letters_accent_classification_test_errors.pkl', 'rb')
syllabled_letters_accent_classification_errors = pickle.load(pickle_input)
pickle_input = open('../accent_classification/ensemble_test_errors.pkl', 'rb')
ensemble_accent_classification_errors = pickle.load(pickle_input)

In [6]:
# TEST DATA
pickle_input = open('letters_word_accetuation_test_error.pkl', 'rb')
letters_word_accetuation = pickle.load(pickle_input)
pickle_input = open('syllables_word_accetuation_test_error.pkl', 'rb')
syllables_word_accetuation = pickle.load(pickle_input)
pickle_input = open('syllabled_letters_word_accetuation_test_error.pkl', 'rb')
syllabled_letters_word_accetuation = pickle.load(pickle_input)
pickle_input = open('ensemble_test_errors.pkl', 'rb')
ensemble_word_accetuation = pickle.load(pickle_input)

pickle_input = open('letters_word_accetuation_test_predictions.pkl', 'rb')
letters_word_accetuation_predictions = pickle.load(pickle_input)
pickle_input = open('syllables_accent_classification_test_predictions.pkl', 'rb')
syllables_word_accetuation_predictions = pickle.load(pickle_input)
pickle_input = open('syllabled_letters_word_accetuation_test_predictions.pkl', 'rb')
syllabled_letters_word_accetuation_predictions = pickle.load(pickle_input)
pickle_input = open('ensemble_test_predictions.pkl', 'rb')
ensemble_word_accetuation_predictions = pickle.load(pickle_input)

FileNotFoundError: [Errno 2] No such file or directory: 'letters_word_accetuation_test_error.pkl'

In [32]:
1 - len(ensemble_accent_classification_errors)/float(data.x_validate.shape[0])

0.9637232119803769

In [20]:
def prediction_accuracy(stress_location_errors, stress_type_errors, length):
    print(1 - len(stress_location_errors)/float(length))
    print(1 - len(stress_type_errors)/float(length))
    collective_error_num = len(stress_location_errors) + len(stress_type_errors) -\
                            len(compare_outputs(letters_accent_classification_errors, letters_word_accetuation_test))
    print(1 - collective_error_num/float(length))
#print(len(letters_accent_classification_errors))
#print(len(letters_word_accetuation_test))
#len(compare_outputs(letters_accent_classification_errors, letters_word_accetuation_test))
print("letters")
prediction_accuracy(letters_word_accetuation, letters_accent_classification_errors, data.x_validate.shape[0])

print("syllables")
prediction_accuracy(syllables_word_accetuation, syllables_accent_classification_errors, data.x_validate.shape[0])

print("syllabled letters")
prediction_accuracy(syllabled_letters_word_accetuation, syllabled_letters_accent_classification_errors, data.x_validate.shape[0])

print("ensembles")
prediction_accuracy(ensemble_word_accetuation, ensemble_accent_classification_errors, data.x_validate.shape[0])

letters
0.8853970712994725
0.9606248386263878
0.8564789199955738
syllables
0.8567002323780015
0.9399690162664601
0.8071262587141751
syllabled letters
0.879845819040242
0.9598318025893549
0.8501346316993103
ensembles
0.9023643539522703
0.9637232119803769
0.8765445760023607


In [ ]:
y_train
y_validate

In [94]:
count_accents = np.zeros(13)
for i in range(data.y_validate.shape[0]):
    for j in range(data.y_validate.shape[1]):
        if data.y_train[i][j] > 0.0:
            #print('')
            count_accents[int(data.y_validate[i][j])] += 1
for el in count_accents:
    print(el)

37251.0
5360.0
191.0
4553.0
236.0
1724.0
4777.0
20.0
2688.0
481.0
2.0
1314.0
0.0


In [97]:
sum(count_accents)
#data.y_validate.shape[0]

58597.0

In [105]:
data.y_test.shape[0]

54222